## **Q3**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Part A(Sample)

In [ ]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=26f9a34d9a61a0c1482b348a3e33af5fce50eceea9113fd95eb9c09e15edcfd4
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
from datetime import datetime


In [ ]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Fava_Project") \
    .master("local[*]") \
    .getOrCreate()
sc=spark.sparkContext

In [ ]:
df=spark.read.csv('/content/drive/MyDrive/SampleData.csv',header=True)
df.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|  100700853|      283|        9016704|      9016704|               5|       264|2022-01-08 00:00:39|
|     230204|       81|        9017538|      9017538|               5|       161|2022-01-08 00:38:48|
|     631357|       81|        9017538|      9017538|               5|       161|2022-01-08 00:13:50|
|     900164|      283|        9035020|      9035020|               5|       264|2022-01-08 00:27:38|
|     900149|      283|        9020900|      9020900|               5|       264|2022-01-08 00:40:08|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 5 rows



In [ ]:
df=df.withColumn("date",to_date("PASS_DAY_TIME"))
df.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+----------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|      date|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+----------+
|  100700853|      283|        9016704|      9016704|               5|       264|2022-01-08 00:00:39|2022-01-08|
|     230204|       81|        9017538|      9017538|               5|       161|2022-01-08 00:38:48|2022-01-08|
|     631357|       81|        9017538|      9017538|               5|       161|2022-01-08 00:13:50|2022-01-08|
|     900164|      283|        9035020|      9035020|               5|       264|2022-01-08 00:27:38|2022-01-08|
|     900149|      283|        9020900|      9020900|               5|       264|2022-01-08 00:40:08|2022-01-08|
+-----------+---------+---------------+-------------+----------------+----------+---------------

In [ ]:
df_rdd=df.rdd.map(lambda x: ((x[3],str(x[7])),x[0]))  # (key=(plate,date),value=DEVICE_CODE)

In [ ]:
df_rdd.take(5)

[(('9016704', '2022-01-08'), '100700853'),
 (('9017538', '2022-01-08'), '230204'),
 (('9017538', '2022-01-08'), '631357'),
 (('9035020', '2022-01-08'), '900164'),
 (('9020900', '2022-01-08'), '900149')]

In [ ]:
rdd_1=df_rdd.groupByKey().mapValues(set).mapValues(list)  #(key=(plate,date),value = [Device Code List]) 

In [ ]:
rdd_1.take(5)

[(('9819872', '2022-01-08'), ['100701260', '900218', '107301', '175']),
 (('9835325', '2022-01-08'), ['900212']),
 (('10717123', '2022-01-08'), ['100701147']),
 (('96661197', '2022-01-08'), ['22010047', '22010135']),
 (('14835536', '2022-01-08'), ['22010112', '22010110'])]

part A(all Data) 

In [ ]:
df_a=spark.read.csv('/content/drive/MyDrive/TrafficData.csv',header=True)
df_a.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|   22010122|      284|       97955760|     64111706|               7|       161|2022-01-10 08:58:02|
|   22010047|      284|       34111333|     34111333|               6|       161|2022-01-10 08:09:03|
|     206701|       81|       33797174|     64111706|               7|       161|2022-01-10 07:50:48|
|   22010053|      284|       33845128|     33845128|               6|       161|2022-01-10 10:53:24|
|     900214|      283|       17144642|     64111706|               7|       264|2022-01-10 08:12:33|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 5 rows



In [ ]:
df_a=df_a.withColumn("date",to_date("PASS_DAY_TIME"))
df_a.show(5)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+----------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|      date|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+----------+
|   22010122|      284|       97955760|     64111706|               7|       161|2022-01-10 08:58:02|2022-01-10|
|   22010047|      284|       34111333|     34111333|               6|       161|2022-01-10 08:09:03|2022-01-10|
|     206701|       81|       33797174|     64111706|               7|       161|2022-01-10 07:50:48|2022-01-10|
|   22010053|      284|       33845128|     33845128|               6|       161|2022-01-10 10:53:24|2022-01-10|
|     900214|      283|       17144642|     64111706|               7|       264|2022-01-10 08:12:33|2022-01-10|
+-----------+---------+---------------+-------------+----------------+----------+---------------

In [ ]:
df_a_rdd=df.rdd.map(lambda x: ((x[3],str(x[7])),x[0]))  # (key=(plate,date),value=DEVICE_CODE)

In [ ]:
df_a_rdd.take(5)

[(('9016704', '2022-01-08'), '100700853'),
 (('9017538', '2022-01-08'), '230204'),
 (('9017538', '2022-01-08'), '631357'),
 (('9035020', '2022-01-08'), '900164'),
 (('9020900', '2022-01-08'), '900149')]

In [ ]:
rdd_a_1=df_a_rdd.groupByKey().mapValues(set).mapValues(list) #(key=(plate,date),value = [Device Code List])

In [ ]:
rdd_a_1.take(10)

[(('9385850', '2022-01-08'), ['900244', '100700839', '900212', '900222']),
 (('9938579', '2022-01-08'),
  ['123',
   '100700820',
   '900266',
   '100700853',
   '900243',
   '900164',
   '230204',
   '631719',
   '631367',
   '119']),
 (('9492185', '2022-01-08'), ['22010139', '22010110']),
 (('11469363', '2022-01-08'), ['22010111', '22010138']),
 (('35558776', '2022-01-08'), ['22010111', '22010110', '22010138']),
 (('35409091', '2022-01-08'), ['22010047']),
 (('8129856', '2022-01-08'), ['900234', '205802', '631354', '900246']),
 (('8515533', '2022-01-08'),
  ['900142', '100700853', '631506', '900212', '631634']),
 (('8572367', '2022-01-08'),
  ['100700804',
   '900142',
   '631368',
   '100700864',
   '100700853',
   '209103',
   '101301',
   '22010117',
   '900218',
   '900197',
   '900164',
   '900185',
   '900129',
   '203902',
   '900212',
   '900273',
   '163',
   '22010054']),
 (('29143159', '2022-01-08'), ['22010047'])]

Part B

In [ ]:
uniq_cam=df.select('DEVICE_CODE').distinct().collect()                            

In [ ]:
import numpy as np

In [ ]:
ind = np.random.randint(0,len(uniq_cam),4) #creating random index for choosing random cameras
ind

array([423, 892, 663, 749])

In [ ]:
hypo_path = np.array(uniq_cam)[(ind).astype(int)] #creating a hypothetical path
hypo_path

array([['22010058'],
       ['631702'],
       ['22009911'],
       ['109']], dtype='<U9')

In [ ]:
rdd_similar_path=rdd_1.filter(lambda x: set(hypo_path[0]).issubset(set(x[1])) or set(hypo_path[1]).issubset(set(x[1])) or set(hypo_path[2]).issubset(set(x[1])) or set(hypo_path[3]).issubset(set(x[1])))

In [ ]:
rdd_similar_path.count()

2563

In [ ]:
rdd_similar_path.take(5)

[(('15537551', '2022-01-08'), ['22010057', '22010058']),
 (('7967132', '2022-01-08'), ['22010057', '22010058']),
 (('12206868', '2022-01-08'), ['22010057', '22010058']),
 (('17079637', '2022-01-08'), ['22010057', '900141', '22010058']),
 (('8469491', '2022-01-08'), ['100701144', '109', '108', '200301'])]

In [ ]:
set_hypo=set()
for i in range(len(hypo_path)):
    set_hypo.add(hypo_path.tolist()[i][0])

In [ ]:
#( key =cos(theta), value = ((plate,date),[Device Code List]) )
rdd_cosine_similarity=rdd_similar_path.map(lambda x: ((len(set_hypo.intersection(set(x[1])))/(np.sqrt(len(set_hypo)) * np.sqrt(len(set(x[1]))) )).round(3),(x[0],x[1])) )

In [ ]:
sorted(rdd_cosine_similarity.collect(),reverse=True)

part B(all data)

In [ ]:
uniq_cam=df_a.select('DEVICE_CODE').distinct().collect()  

In [ ]:
len(uniq_cam)

946

In [ ]:
ind = np.random.randint(0,len(uniq_cam),4) #creating random index for choosing random camera
ind

array([177, 127, 354, 297])

In [ ]:
hypo_path = np.array(uniq_cam)[(ind).astype(int)] #creating a hypothetical path
hypo_path

array([['22009806'],
       ['1001001'],
       ['900195'],
       ['100701298']], dtype='<U9')

In [ ]:
rdd_similar_path=rdd_a_1.filter(lambda x: set(hypo_path[0]).issubset(set(x[1])) or set(hypo_path[1]).issubset(set(x[1])) or set(hypo_path[2]).issubset(set(x[1])) or set(hypo_path[3]).issubset(set(x[1])))

In [ ]:
rdd_similar_path.count()

3953

In [ ]:
rdd_similar_path.take(5)

[(('42083983', '2022-01-08'), ['22010122', '100701298', '22010054']),
 (('60482918', '2022-01-08'), ['100701298', '22010125']),
 (('20250397', '2022-01-08'), ['100701298']),
 (('26108324', '2022-01-08'), ['100701298', '22010127']),
 (('91978972', '2022-01-08'), ['100701298'])]

In [ ]:
set_hypo=set()
for i in range(len(hypo_path)):
    set_hypo.add(hypo_path.tolist()[i][0])

In [ ]:
#( key =cos(theta), value = ((plate,date),[Device Code List]) )
rdd_cosine_similarity=rdd_similar_path.map(lambda x: ((len(set_hypo.intersection(set(x[1])))/(np.sqrt(len(set_hypo)) * np.sqrt(len(set(x[1]))) )).round(3),(x[0],x[1])) )

In [ ]:
sorted(rdd_cosine_similarity.take(15),reverse=True)

[(0.5, (('91978972', '2022-01-08'), ['100701298'])),
 (0.5, (('74609983', '2022-01-08'), ['100701298'])),
 (0.5, (('40201285', '2022-01-08'), ['100701298'])),
 (0.5, (('20250397', '2022-01-08'), ['100701298'])),
 (0.354, (('9423700', '2022-01-08'), ['22010121', '100701298'])),
 (0.354, (('88467288', '2022-01-08'), ['22010112', '100701298'])),
 (0.354, (('8738634', '2022-01-08'), ['100701298', '22010128'])),
 (0.354, (('60482918', '2022-01-08'), ['100701298', '22010125'])),
 (0.354, (('26108324', '2022-01-08'), ['100701298', '22010127'])),
 (0.354, (('20211489', '2022-01-08'), ['100701298', '22010059'])),
 (0.289,
  (('60123951', '2022-01-08'), ['100701298', '100701119', '100701297'])),
 (0.289, (('42083983', '2022-01-08'), ['22010122', '100701298', '22010054'])),
 (0.25,
  (('26173221', '2022-01-08'),
   ['100701298', '900202', '900155', '100701157'])),
 (0.224,
  (('19745155', '2022-01-08'),
   ['205202', '205201', '169', '22009972', '100701298'])),
 (0.204,
  (('104633602', '2022-01-

**part C**